<a href="https://colab.research.google.com/github/varunsawhney8/Projects/blob/main/sample20Aug.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
print(round(0.665, 1))

0.7


In [4]:
import pandas as pd
import numpy as np


path='/content/Reliance_Final_Ndtv_data4.xlsx'
data=pd.read_excel(path)


data=data[::-1].reset_index()
data.drop(columns=['index'],axis=True,inplace=True)

path='/content/Reliance_Nse_data2.csv'
data1=pd.read_csv(path)


data['Date']=data['Date'].astype('str')

data["Price"]=[np.nan for i in range(data.shape[0])]




for i in range(len(data['Date'])):
    for j in range(len(data1['Date'])):
        if data['Date'][i]==data1['Date'][j]:
            data["Price"][i]=data1["Close"][j]
        

data["Price"]=data["Price"].fillna(method='bfill')

data["Price"][372:]=2174



# =============================================================================
# # Simple Moving Average
# 
# data1['SMA_3'] = data1.iloc[:,1].rolling(window=3).mean().round()
# 
# # Weighted Moving Average
# 
# weights = np.arange(1,11)
# wma10 = data1['Close'].rolling(10).apply(lambda prices: np.dot(prices, weights)/weights.sum(), raw=True)
# 
# 
# =============================================================================
# Exponential Moving Average

#data['7day_ea'] = data['Price'].ewm(span=7).mean()

data.columns
data.drop(columns=['Media','Article','Keywords'],inplace=True,axis=1)


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [5]:
data.columns

Index(['Date', 'Title', 'Price'], dtype='object')

In [6]:
!pip install transformers
from transformers import pipeline
classifier = pipeline('sentiment-analysis', model="ProsusAI/finbert")
list3=[]

for i in range(len(data['Title'])):
    a=(classifier(data['Title'][i]))
    a=a[0]
    a=list(a.values())[0]

    if a=='positive':
        a=1
        list3.append(a)
    elif a=='negative' :
        a=2
        list3.append(a)
    else:
        a=0
        list3.append(a)

data['finbert']=list3

del list3


     |████████████████████████████████| 2.6 MB 4.1 MB/s 
     |████████████████████████████████| 3.3 MB 39.3 MB/s 
     |████████████████████████████████| 895 kB 73.8 MB/s 
     |████████████████████████████████| 636 kB 60.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


Downloading:   0%|          | 0.00/758 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/438M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/252 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
data.head()

,Date,Title,Price,finbert
0,2020-07-16,Reliance's Stake Sale Talks With Saudi Aramco ...,1843.400024,2
1,2020-07-17,Reliance To Shut Crude Refining Unit For Maint...,1911.699951,2
2,2020-07-20,Reliance-Google Smartphone Deal in India Threa...,1919.900024,2
3,2020-07-21,"Netflix, Amazon, Facebook Stocks Draw In Amate...",1971.550049,1
4,2020-07-21,JioMart App for Android and iOS Launched: Here...,1971.550049,0


In [7]:
aggregation_functions = {'Price': 'mean', 'Title': 'first', 'finbert': 'max'}
data = data.groupby(data['Date']).aggregate(aggregation_functions)

In [8]:
#data.head()
data['7day_sentiment'] = round(data['finbert'].ewm(span=7).mean())

In [12]:
data.columns

Index(['Price', 'Title', 'finbert', '7day_sentiment', 'perc_change'], dtype='object')

In [10]:
data.shape

(175, 5)

In [9]:
data['perc_change'] =data['Price'].pct_change(periods=1)*100

In [13]:
data.index

Index(['2020-07-16', '2020-07-17', '2020-07-20', '2020-07-21', '2020-07-22',
       '2020-07-23', '2020-07-24', '2020-07-26', '2020-07-27', '2020-07-29',
       ...
       '2021-07-29', '2021-07-30', '2021-08-02', '2021-08-03', '2021-08-04',
       '2021-08-06', '2021-08-08', '2021-08-10', '2021-08-11', '2021-08-16'],
      dtype='object', name='Date', length=175)

In [77]:
def strategy_eval(data,n):
  profit=0
  loss=0
  neutral=0
  no_sig=0
  a=['finbert','7day_sentiment']
  p=['perc_change']
  if n==1:
    val=a[0]
    per=p[0]
  else:
      val=a[1]
      per=p[1]
  for i in range(len(data)-n):
      if data[val][i]==2:
        if data[per][i+n]<0:
          profit+=1
        elif data[per][i+n]==0:
          neutral+=1
        else:
            loss+=1
      if data[val][i]==1:
        if data[per][i+n]<0:
          loss+=1
        elif data[per][i+n]==0:
          neutral+=1
        else:
          profit+=1
      if data[val][i]==0:
        no_sig+=1
   
  return profit,loss,neutral,no_sig



In [79]:
p,l,n,no=strategy_eval(data,7)

In [76]:
p,l,n,no

(79, 63, 24, 2)

In [83]:
data.head(50)

,Price,Title,finbert,7day_sentiment,perc_change,7day_perc
Date,,,,,,
2020-07-16,1843.400024,Reliance's Stake Sale Talks With Saudi Aramco ...,2,2.0,NaN,NaN
2020-07-17,1911.699951,Reliance To Shut Crude Refining Unit For Maint...,2,2.0,3.705106,NaN
2020-07-20,1919.900024,Reliance-Google Smartphone Deal in India Threa...,2,2.0,0.428941,NaN
2020-07-21,1971.550049,"Netflix, Amazon, Facebook Stocks Draw In Amate...",1,2.0,2.690246,NaN
2020-07-22,2004.000000,Reliance Industries Postpones June Quarter Res...,2,2.0,1.645911,NaN
2020-07-23,2057.800049,"Sensex, Nifty Resume Upmove After A Day's Brea...",1,2.0,2.684633,NaN
2020-07-24,2146.149902,Sensex Registers Sixth Weekly Gain In A Row As...,1,1.0,4.293413,NaN
2020-07-26,2156.199951,India June Oil Imports Lowest In Nearly Nine Y...,2,2.0,0.468283,16.968641
2020-07-27,2156.199951,Mukesh Ambani's Reliance Becomes World's No. 2...,1,1.0,0.000000,12.789664
